In [ ]:
import numpy as np
import pandas as pd
import os
import sklearn
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import roc_curve
from sklearn.metrics import auc
from sklearn.utils import shuffle
import rpy2.robjects as ro
from rpy2.robjects.packages import importr, data
from rpy2.robjects import pandas2ri
from rpy2.robjects import numpy2ri
from tabulate import tabulate
import scipy
from cliffs_delta import cliffs_delta
import seaborn as sns


In [ ]:
import warnings
warnings.filterwarnings("ignore")

In [ ]:
%load_ext rpy2.ipython

In [ ]:
utils = importr('utils')
base = importr('base')
utils.chooseCRANmirror(ind=1)

In [ ]:
def arrow(num):
    return '↗' if num > 0 else '↘'
def signf_code(chisq):
    if chisq <= 0.001:
        return '***'
    elif chisq <= 0.01:
        return '**'
    elif chisq <= 0.05:
        return '*'
    elif chisq <= 0.1:
        return '.'
    else:
        return ' '

In [ ]:
def min_max_normalize(column):
    col_min = column.min()
    col_max = column.max()
    if col_min == col_max:
        return 0 if col_min == 0 else column / col_min
    else:
        return (column - col_min) / (col_max - col_min)

## Prepare training and testing dataset

In [ ]:
basedir = os.path.join("..","data")
developer_data = pd.read_csv(os.path.join(basedir, 'contributor_features.csv'), index_col = 0)

In [ ]:
features = ['Duration', 'Timezone', 'Worktime',
       'Total Commits', 'Authored files', 'Commit Rate', 'Code Commits',
       'Code Commit Rate', 'Other Commits', 'Other Commit Rate',
       'Code Contribution', 'Code Contribution Rate',
       'Code Contribution Density', 'Total Issues', 'Issue Participated',
       'Issue Solved', 'Issue Contribution', 'Issue Contribution Rate',
       'Issue Solving Ratio', 'Issue Solving Density', 'Total Pull Requests',
       'PR Merged', 'PR Reviewed', 'PR Participated', 'PR Contribution',
       'PR Contribution Rate', 'PR Approval Ratio', 'PR Approval Density',
       'Followers', 'Collaborations', 'Languages']

non_correlated_redundant = ['Duration','Timezone','Worktime','Total Commits','Authored files','Commit Rate','Code Commit Rate','Other Commits',
                             'Code Contribution Rate','Code Contribution Density','Total Issues', 'Issue Contribution Rate','Issue Solved',
                             'Issue Participated','PR Merged','PR Reviewed','PR Contribution Rate','PR Approval Ratio','PR Approval Density','Followers',
                              'Collaborations','Languages']

In [ ]:
clustering_features = ["Worktime","Code Contribution Density", "Languages", "Total Commits"]
non_cluster_features = [item for item in features if not item in clustering_features]
noncor_model_features = [item for item in non_correlated_redundant if not item in clustering_features]

In [ ]:
encoder = sklearn.preprocessing.LabelEncoder()
#encoder.fit([['tensorflow', 0], ['pytorch', 1], ['keras', 2], ['mxnet', 3], ['theano',4], ['onnx', 5]])
encoder.fit(developer_data["Project"])
developer_data["project_encoded"]=encoder.transform(developer_data["Project"])
developer_data["project_encoded"].value_counts()

In [ ]:
# normalization
normalized = developer_data.copy(deep=True)
for fea in noncor_model_features:
    normalized[fea] = min_max_normalize(normalized[fea])

## Binary logistic regression

In [ ]:
# run this cell to install required packages only for first time executing this file
utils.install_packages('rms')
utils.install_packages('stats')
utils.install_packages('ScottKnottESD')
utils.install_packages('Hmisc')
utils.install_packages('car')
utils.install_packages('lmtest')
utils.install_packages('pROC')
utils.install_packages('caret')

In [ ]:
rms = importr('rms')
lmtest = importr('lmtest')
stats = importr('stats')
car = importr('car')
hmisc = importr('Hmisc')
proc = importr('pROC')
caret = importr('caret')
graphics = importr('graphics')
sk = importr('ScottKnottESD')
#grdevices = importr('grDevices')

In [ ]:
# active pandas dataframe and numpy array in r
pandas2ri.activate()
numpy2ri.activate()

### profile 0 : core - afterhour

In [ ]:
# the result might be slightly different every time executing thie code due to data sampling and shuffling
profile = 'ca'
developer_data.loc[developer_data['profile'] == profile, 'model_label'] = 1
developer_data.loc[developer_data['profile'] != profile, 'model_label'] = 0

total = developer_data['model_label'].value_counts()[0]
numerator = developer_data['model_label'].value_counts()[1]
dataset = shuffle(pd.concat([developer_data.loc[developer_data['model_label'] == 1],
                    developer_data.loc[developer_data['model_label'] == 0].sample(developer_data['model_label'].value_counts()[1])]))
current_features = noncor_model_features#+["project_encoded"]
x = np.array(dataset[current_features])
y = np.array(dataset['model_label'])

current_features = [item.replace(' ','.') for item in current_features]
dataset = pd.DataFrame(data=x, columns = [item.replace(' ','.') for item in current_features])
dataset['model_label'] = y


with (ro.default_converter + pandas2ri.converter).context():
  dataset_r = ro.conversion.py2rpy(dataset)

frmla = 'model_label' + ' ~ ' +' + '.join(dataset.columns[:-1])
frmla
# ddist = rms.datadist(dataset_r)
# base.options(datadist=ddist)

glm_model = stats.glm(formula=ro.Formula(frmla), data=dataset_r,maxit=1000)
print('glm model done')
step_glm = stats.step(glm_model, direction='both', trace='F')
w = car.Anova(step_glm, type="II", test="Wald")
coef = lmtest.coeftest(step_glm)
with (ro.default_converter + pandas2ri.converter).context():
  w_pd = ro.conversion.get_conversion().rpy2py(w)
with (ro.default_converter + pandas2ri.converter).context():
  step_glm = ro.conversion.get_conversion().rpy2py(step_glm)
fitted_values = list(map(lambda x: 0 if x < 0.5 else 1, step_glm['fitted.values']))
# roc1 <- roc(data1$lr_label, m1$fitted.values)
# auc(roc1)
accuracy = accuracy_score(y, fitted_values)
fpr, tpr, thresholds = roc_curve(y, fitted_values)
_auc = auc(fpr, tpr)
print('number of datapoints: ',len(dataset))
print('Accuracy: ',accuracy)
print('AUC: ',_auc)

w_pd['Chisq%'] = w_pd['Chisq']*100/w_pd['Chisq'].sum()
w_pd['Chisq%'] = w_pd['Chisq%'].round(2)
w_pd['Signf.'] = w_pd['Pr(>Chisq)'].apply(signf_code)
w_pd['Rel.'] = list(zip(*coef))[0][1:]
w_pd['Rel.'] = w_pd['Rel.'].apply(arrow)
w_pd[['Chisq%','Signf.', 'Rel.']]

### profile 1 : core - workhour

In [ ]:
# active pandas dataframe and numpy array in r
profile = 'cw'
developer_data.loc[developer_data['profile'] == profile, 'model_label'] = 1
developer_data.loc[developer_data['profile'] != profile, 'model_label'] = 0
dataset = shuffle(pd.concat([developer_data.loc[developer_data['model_label'] == 1],
                    developer_data.loc[developer_data['model_label'] == 0].sample(developer_data['model_label'].value_counts()[1])]))
current_features = noncor_model_features#+["project_encoded"]
x = np.array(dataset[current_features])
y = np.array(dataset['model_label'])

current_features = [item.replace(' ','.') for item in current_features]
dataset = pd.DataFrame(data=x, columns = [item.replace(' ','.') for item in current_features])
dataset['model_label'] = y


with (ro.default_converter + pandas2ri.converter).context():
  dataset_r = ro.conversion.py2rpy(dataset)

frmla = 'model_label' + ' ~ ' +' + '.join(dataset.columns[:-1])
frmla
# ddist = rms.datadist(dataset_r)
# base.options(datadist=ddist)

glm_model = stats.glm(formula=ro.Formula(frmla), data=dataset_r,maxit=1000)
step_glm = stats.step(glm_model, direction='both', trace='F')
w = car.Anova(step_glm, type="II", test="Wald")
coef = lmtest.coeftest(step_glm)
with (ro.default_converter + pandas2ri.converter).context():
  w_pd = ro.conversion.get_conversion().rpy2py(w)
# fitted_values = ro.r.predict(lrm_model, type='fitted.ind')
# fitted_values = list(map(lambda x: 0 if x < 0.5 else 1, fitted_values))
with (ro.default_converter + pandas2ri.converter).context():
  step_glm = ro.conversion.get_conversion().rpy2py(step_glm)
fitted_values = list(map(lambda x: 0 if x < 0.5 else 1, step_glm['fitted.values']))
# roc1 <- roc(data1$lr_label, m1$fitted.values)
# auc(roc1)
with (ro.default_converter + pandas2ri.converter).context():
  step_glm = ro.conversion.get_conversion().rpy2py(step_glm)
fitted_values = list(map(lambda x: 0 if x < 0.5 else 1, step_glm['fitted.values']))
accuracy = accuracy_score(y, fitted_values)
fpr, tpr, thresholds = roc_curve(y, fitted_values)
_auc = auc(fpr, tpr)
print('number of datapoints: ',len(dataset))
print('Accuracy: ',accuracy)
print('AUC: ',_auc)
print('number of datapoints: ',len(dataset))

w_pd['Chisq%'] = w_pd['Chisq']*100/w_pd['Chisq'].sum()
w_pd['Chisq%'] = w_pd['Chisq%'].round(2)
w_pd['Signf.'] = w_pd['Pr(>Chisq)'].apply(signf_code)
w_pd['Rel.'] = list(zip(*coef))[0][1:]
w_pd['Rel.'] = w_pd['Rel.'].apply(arrow)
w_pd[['Chisq%','Signf.', 'Rel.']]

### profile 2 : peripheral - Afterhour

In [ ]:
# active pandas dataframe and numpy array in r
profile = 'pa'
developer_data.loc[developer_data['profile'] == profile, 'model_label'] = 1
developer_data.loc[developer_data['profile'] != profile, 'model_label'] = 0
dataset = shuffle(pd.concat([developer_data.loc[developer_data['model_label'] == 1],
                    developer_data.loc[developer_data['model_label'] == 0].sample(developer_data['model_label'].value_counts()[1])]))
current_features = noncor_model_features+["project_encoded"]
x = np.array(dataset[current_features])
y = np.array(dataset['model_label'])

current_features = [item.replace(' ','.') for item in current_features]
dataset = pd.DataFrame(data=x, columns = [item.replace(' ','.') for item in current_features])
dataset['model_label'] = y


with (ro.default_converter + pandas2ri.converter).context():
  dataset_r = ro.conversion.py2rpy(dataset)

frmla = 'model_label' + ' ~ ' +' + '.join(dataset.columns[:-1])
frmla
# ddist = rms.datadist(dataset_r)
# base.options(datadist=ddist)

glm_model = stats.glm(formula=ro.Formula(frmla), data=dataset_r,maxit=1000)
step_glm = stats.step(glm_model, direction='both', trace='F')
w = car.Anova(step_glm, type="II", test="Wald")
coef = lmtest.coeftest(step_glm)
with (ro.default_converter + pandas2ri.converter).context():
  w_pd = ro.conversion.get_conversion().rpy2py(w)
# fitted_values = ro.r.predict(lrm_model, type='fitted.ind')
# fitted_values = list(map(lambda x: 0 if x < 0.5 else 1, fitted_values))

with (ro.default_converter + pandas2ri.converter).context():
  step_glm = ro.conversion.get_conversion().rpy2py(step_glm)
fitted_values = list(map(lambda x: 0 if x < 0.5 else 1, step_glm['fitted.values']))
accuracy = accuracy_score(y, fitted_values)
fpr, tpr, thresholds = roc_curve(y, fitted_values)
_auc = auc(fpr, tpr)
print('number of datapoints: ',len(dataset))
print('Accuracy: ',accuracy)
print('AUC: ',_auc)

print('number of datapoints: ',len(dataset))

w_pd['Chisq%'] = w_pd['Chisq']*100/w_pd['Chisq'].sum()
w_pd['Chisq%'] = w_pd['Chisq%'].round(2)
w_pd['Signf.'] = w_pd['Pr(>Chisq)'].apply(signf_code)
w_pd['Rel.'] = list(zip(*coef))[0][1:]
w_pd['Rel.'] = w_pd['Rel.'].apply(arrow)
w_pd[['Chisq%','Signf.', 'Rel.']]

### profile 3 : peripheral - workhour

In [ ]:
# active pandas dataframe and numpy array in r
profile = 'pw'
developer_data.loc[developer_data['profile'] == profile, 'model_label'] = 1
developer_data.loc[developer_data['profile'] != profile, 'model_label'] = 0
dataset = shuffle(pd.concat([developer_data.loc[developer_data['model_label'] == 1],
                    developer_data.loc[developer_data['model_label'] == 0].sample(developer_data['model_label'].value_counts()[1])]))
current_features = noncor_model_features#+["project_encoded"]
x = np.array(dataset[current_features])
y = np.array(dataset['model_label'])

current_features = [item.replace(' ','.') for item in current_features]
dataset = pd.DataFrame(data=x, columns = [item.replace(' ','.') for item in current_features])
dataset['model_label'] = y

with (ro.default_converter + pandas2ri.converter).context():
  dataset_r = ro.conversion.py2rpy(dataset)

frmla = 'model_label' + ' ~ ' +' + '.join(dataset.columns[:-1])
frmla
# ddist = rms.datadist(dataset_r)
# base.options(datadist=ddist)

glm_model = stats.glm(formula=ro.Formula(frmla), data=dataset_r,maxit=1000)
step_glm = stats.step(glm_model, direction='both', trace='F')
w = car.Anova(step_glm, type="II", test="Wald")
coef = lmtest.coeftest(step_glm)
with (ro.default_converter + pandas2ri.converter).context():
  w_pd = ro.conversion.get_conversion().rpy2py(w)
# fitted_values = ro.r.predict(lrm_model, type='fitted.ind')
# fitted_values = list(map(lambda x: 0 if x < 0.5 else 1, fitted_values))
with (ro.default_converter + pandas2ri.converter).context():
  step_glm = ro.conversion.get_conversion().rpy2py(step_glm)
fitted_values = list(map(lambda x: 0 if x < 0.5 else 1, step_glm['fitted.values']))
# roc1 <- roc(data1$lr_label, m1$fitted.values)
# auc(roc1)
with (ro.default_converter + pandas2ri.converter).context():
  step_glm = ro.conversion.get_conversion().rpy2py(step_glm)
fitted_values = list(map(lambda x: 0 if x < 0.5 else 1, step_glm['fitted.values']))
accuracy = accuracy_score(y, fitted_values)
fpr, tpr, thresholds = roc_curve(y, fitted_values)
_auc = auc(fpr, tpr)
print('number of datapoints: ',len(dataset))
print('Accuracy: ',accuracy)
print('AUC: ',_auc)

print('number of datapoints: ',len(dataset))

w_pd['Chisq%'] = w_pd['Chisq']*100/w_pd['Chisq'].sum()
w_pd['Chisq%'] = w_pd['Chisq%'].round(2)
w_pd['Signf.'] = w_pd['Pr(>Chisq)'].apply(signf_code)
w_pd['Rel.'] = list(zip(*coef))[0][1:]
w_pd['Rel.'] = w_pd['Rel.'].apply(arrow)
w_pd[['Chisq%','Signf.', 'Rel.']]

## statistical test

In [ ]:
ca = developer_data.loc[developer_data['profile']=='ca']
cw = developer_data.loc[developer_data['profile']=='cw']
pa = developer_data.loc[developer_data['profile']=='pa']
pw = developer_data.loc[developer_data['profile']=='pw']
core = developer_data.loc[developer_data['profile'].isin(['ca','cw'])]
peri = developer_data.loc[developer_data['profile'].isin(['pa','pw'])]

In [ ]:
# compare core and peripheral contributors
for fea in non_cluster_features:
    x1, x2 = core[fea],peri[fea]
    stat, pval = scipy.stats.mannwhitneyu(x1, x2)
    d, res = cliffs_delta(x1, x2)
    if pval < 0.05: # significantly different
        print(fea)
        print(stat, pval)
        print(d, res)
        print('-------------------------------------------------')
    else:  # not significantly different
        print(fea)
        print('not significant')
        print('-------------------------------------------------')

In [ ]:
# compare core-afterhour and core-workhour contributors
for fea in non_cluster_features:
    x1, x2 = ca[fea],cw[fea]
    stat, pval = scipy.stats.mannwhitneyu(x1, x2)
    d, res = cliffs_delta(x1, x2)
    if pval < 0.05: # significantly different
        print(fea)
        print(stat, pval)
        print(d, res)
        print('-------------------------------------------------')
    else:  # not significantly different
        print(fea)
        print('not significant')
        print('-------------------------------------------------')


In [ ]:
# compare peripheral-afterhour and peripheral-workhour contributors
for fea in non_cluster_features:
    x1, x2 = pa[fea],pw[fea]
    stat, pval = scipy.stats.mannwhitneyu(x1, x2)
    d, res = cliffs_delta(x1, x2)
    if pval < 0.05: # significantly different
        print(fea)
        print(stat, pval)
        print(d, res)
        print('-------------------------------------------------')
    else:  # not significantly different
        print(fea)
        print('not significant')
        print('-------------------------------------------------')